# Assignment – Build the same interactive ML Model for IPL final 2023 – GT vs CSK

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('max_colwidth',-1)

# Fetching Probability Values:

In [2]:
t20 = pd.read_csv('all_matches.csv')

t20['total_runs'] = t20.apply(lambda x : x['runs_off_bat'] + x['extras'],axis=1)

t20['isOut'] = t20['player_dismissed'].apply(lambda x : 1 if type(x) == type('str') else 0)

t20_dff = t20.sort_values(['match_id','innings','ball'], ascending=[True,True,True]).reset_index(drop=True)

t1 = 'Gujarat Titans'
t2 = 'Chennai Super Kings'

t1_outcome_counts = t20_dff[t20_dff.batting_team==t1].total_runs.value_counts()
t2_outcome_counts = t20_dff[t20_dff.batting_team==t2].total_runs.value_counts()

t1_outs = t20_dff[t20_dff.batting_team==t1].isOut.sum()
t2_outs = t20_dff[t20_dff.batting_team==t2].isOut.sum()

# 0,1,2,3,4,5,6,7,8,w
outcomes = [0,1,2,3,4,6,'w']

t1_outcomes = []
t2_outcomes = []

for outcome in outcomes:
    if outcome != 'w':
        t1_outcomes.append(t1_outcome_counts[outcome])
        t2_outcomes.append(t2_outcome_counts[outcome])
    else:
        t1_outcomes.append(t1_outs)
        t2_outcomes.append(t2_outs)

t1_pb_outcomes = [i/sum(t1_outcomes) for i in t1_outcomes]
t2_pb_outcomes = [i/sum(t2_outcomes) for i in t2_outcomes]

t1_pb_ls = np.cumsum(t1_pb_outcomes)
t2_pb_ls = np.cumsum(t2_pb_outcomes)

In [3]:
t1_pb_ls,t2_pb_ls

(array([0.28958084, 0.71976048, 0.78610778, 0.78826347, 0.91137725,
        0.95976048, 1.        ]),
 array([0.32709314, 0.73127942, 0.7968906 , 0.80067038, 0.90867922,
        0.95835116, 1.        ]))

## Run Prediction Function:

In [4]:
def run_prediction(current_score,current_wickets,current_overs,target):

    pred_runs = current_score
    pred_wickets = current_wickets
    overs = current_overs
    
    leftover_balls = 90 - current_overs*6

    i2_p0 = t2_pb_ls[0]
    i2_p1 = t2_pb_ls[1]
    i2_p2 = t2_pb_ls[2]
    i2_p3 = t2_pb_ls[3]
    i2_p4 = t2_pb_ls[4]
    i2_p6 = t2_pb_ls[5]
    i2_pw = t2_pb_ls[6]

    for i in range(leftover_balls):
        r_value = np.random.random()

        if r_value <= i2_p0:
            pred_runs += 0
        elif r_value <= i2_p1:
            pred_runs += 1
        elif r_value <= i2_p2:
            pred_runs += 2
        elif r_value <= i2_p3:
            pred_runs += 3
        elif r_value <= i2_p4:
            pred_runs += 4
        elif r_value <= i2_p6:
            pred_runs += 6
        else:
            pred_runs += 0
            pred_wickets += 1

            if (pred_wickets == 10):
                break
                
        if pred_runs > target:
            break
            
    return pred_runs

## Get Win Function:

In [5]:
def get_win(pred_runs, target):
    if pred_runs > target:
        return 'win'
    elif pred_runs == target:
        return 'tie'
    else:
        return 'lose'

# Win Count from Run Prediction:

In [6]:
# runs, wickets, overs, 
# win - 1st ing score

# GT - 214/4
# 171 in 15 overs Due to DLS
target = 171

current_score = 7
current_wickets = 78
current_overs = 2

runs_ls = []
results_ls = []

win_count = 0
tie_count = 0
lose_count = 0


for i in range(100):
    pred_runs = run_prediction(current_score,current_wickets,current_overs,target)
    runs_ls.append(pred_runs)
    result_pred = get_win(pred_runs, target)
    results_ls.append(result_pred)

    if result_pred == 'win':
        win_count += 1
    elif result_pred == 'tie':
        tie_count += 1
    else:
        lose_count +=1

In [7]:
win_count, tie_count, lose_count

(0, 0, 100)

## Find out Runs at a required stage:

In [8]:
def find_runs(current_score,current_wickets,current_overs,target):
    
    runs_ls = []
    results_ls = []
    win_ls = []
    req_runs = []
    
    for i in range(current_score,target+1):
        win_count = 0
        tie_count = 0
        lose_count = 0
        
        for j in range(100):
            pred_runs = run_prediction(i, current_wickets, current_overs, target)
            runs_ls.append(pred_runs) 
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)

            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count += 1    
            
            win_ls.append(win_count)
            req_runs.append(i)
    
    required_runs = current_score
    for i in range(len(req_runs)):
        if win_ls[i] >= 50:
            required_runs = req_runs[i]
            break
            
    return required_runs

In [9]:
find_runs(78,2,7,171)

112

## Find out Wickets they can afford to lose:

In [10]:
def find_wickets(current_score,current_wickets,current_overs,target):
    
    runs_ls = []
    results_ls = []
    win_ls = []
    req_wkts = []
    
    for i in range(current_wickets,10):
        win_count = 0
        tie_count = 0
        lose_count = 0
        
        for j in range(100):
            pred_runs = run_prediction(current_score, i, current_overs, target)
            runs_ls.append(pred_runs) 
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)

            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count += 1    
            
        win_ls.append(win_count)
        req_wkts.append(i)
    
    required_wkts = current_wickets
    for i in range(len(req_wkts)):
        if win_ls[i] < 45:
            required_wkts = req_wkts[i]
            break
            
    return required_wkts

In [11]:
find_wickets(78,2,7,171)

2

## Function: Interactive chart

In [12]:
%matplotlib inline
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

def find_runs_wickets(current_wks, at_overs, target_score):
    plt.figure(figsize = (16, 6))
    # x = np.linspace(-10, 10, num=1000)
    x = np.array(list(range(16)))
    req_value = find_runs(78, current_wks, at_overs, target_score)
    req_wk_value = find_wickets(78, current_wks, at_overs, target_score)

    if at_overs == 7:
        req_value = 78
        req_wk_value = 2

    
    # print(req_value)
    y = np.array([req_value for i in range(16)])

    # plt.plot(x, current_overs * x + target_score)
    # plt.plot(x, y)
    plt.scatter(at_overs, req_value, s = 1500, color = 'y')
    plt.axhline(target_score, ls = '--', color = 'blue')
    plt.text( 1, target_score + 10, 'Target Score :' + str(target_score) , color = 'darkblue', fontsize = 13)
    plt.text( at_overs, req_value, str(req_value) + '/' + str(req_wk_value), color = 'white', fontsize = 12,  horizontalalignment='center', verticalalignment='center')
    plt.text(at_overs, req_value - 20, 'CSK has to be at ' + str(req_value) + '/' +  str(req_wk_value) + ' after ' + str(at_overs) + ' overs', horizontalalignment='center')
    plt.ylim(50, target_score + 50)
    plt.xticks(x)
    plt.title('Where should CSK be?', fontsize = 20)
    plt.xlabel('Overs')
    plt.ylabel('Score')
    plt.show()

# x=widgets.IntSlider(min=-10, max=30, step=1, value=10)

# find_wickets(current_score, target, current_wickets, at_overs)
# find_wickets(87, 167, 1, 14)


## Interactive Visualization:

In [13]:
print('current_score = IND: 78/2 (7 overs)')
print('')

interactive_plot = interactive(find_runs_wickets, current_wks = widgets.IntSlider(min=1, max=10, step=1, value=2),  at_overs=widgets.IntSlider(min=7, max=15, step=1, value=7), target_score = widgets.IntSlider(min=0, max=250, step=1, value=171))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

current_score = IND: 78/2 (7 overs)



interactive(children=(IntSlider(value=2, description='current_wks', max=10, min=1), IntSlider(value=7, descrip…